# Tóm tắt tin tức từ all_news.csv

Notebook này sử dụng **Google Gemini AI** và **TOON format** để:
- 📰 Tóm tắt tin tức tài chính từ `all_news.csv`
- 🎯 Tự động nhận diện mã cổ phiếu
- 💰 **Tiết kiệm 30-60% token** nhờ TOON format
- ⚡ Tăng tốc độ phản hồi và giảm chi phí API

---

## 🎯 Điểm nổi bật

✨ **TOON Format Integration** - Định dạng tối ưu cho LLM, giảm token hiệu quả  
🔄 **Retry Logic** - Tự động thử lại khi gặp lỗi  
💾 **Checkpoint System** - Lưu tiến độ, tiếp tục được khi bị gián đoạn  
🤖 **Multi-model Strategy** - Tự động chuyển model khi cần  
📊 **Stock Code Detection** - Nhận diện thông minh mã cổ phiếu từ nội dung

In [27]:
# Import các thư viện cần thiết
import pandas as pd
import os
import time
import re
import json
import subprocess
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
import google.generativeai as genai

# Load environment variables
load_dotenv()

# Cấu hình API Keys với rotation support
api_keys_str = os.getenv("GOOGLE_API_KEY", "")
GOOGLE_API_KEYS = [key.strip() for key in api_keys_str.split(",") if key.strip()]

if not GOOGLE_API_KEYS:
    raise ValueError("GOOGLE_API_KEY không được cấu hình")

# Khởi tạo với key đầu tiên
current_key_index = 0
current_api_key = GOOGLE_API_KEYS[current_key_index]
genai.configure(api_key=current_api_key)

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.02)
model_more_temperature = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.1)
model_pro = ChatGoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25", temperature=0.1)

print(f"Đã khởi tạo models với {len(GOOGLE_API_KEYS)} API key(s)")
print(f"Đang dùng key: ...{current_api_key[-8:]}")

Đã khởi tạo models với 5 API key(s)
Đang dùng key: ...JIlHSfS4


## 📋 Cấu trúc dữ liệu

File `all_news.csv` có các cột sau:
- `postID`: ID bài viết
- `date`: Ngày xuất bản
- `sentiment`: Phân tích cảm xúc (nếu có)
- `taggedSymbols`: Danh sách mã cổ phiếu được tag (dạng list)
- `combinedContent`: Nội dung bài báo đầy đủ (bao gồm Title và Description)

In [28]:
# Test: Đọc và kiểm tra dữ liệu
def test_read_news():
    """
    Kiểm tra xem có đọc được file all_news.csv không
    """
    try:
        df_test = pd.read_csv("all_news/all_news.csv", nrows=3)
        print(f"✓ Đọc thành công file all_news.csv")
        print(f"\n📊 Các cột: {df_test.columns.tolist()}")
        print(f"\n📝 Mẫu dữ liệu (3 dòng đầu):")
        print(df_test[['postID', 'date', 'taggedSymbols']].head())
        
        # Kiểm tra combinedContent
        if len(df_test) > 0:
            print(f"\n📄 Mẫu combinedContent:")
            print(df_test.iloc[0]['combinedContent'][:200] + "...")
        
        return True
    except Exception as e:
        print(f"❌ Lỗi khi đọc file: {e}")
        return False

# Chạy test (bỏ comment để kiểm tra)
# test_read_news()

In [29]:
# Định nghĩa prompt template tối ưu với TOON format
news_summarize_with_stock_template = PromptTemplate.from_template("""
Bạn là một chuyên gia phân tích thị trường chứng khoán Việt Nam. 

Dữ liệu đầu vào có thể ở định dạng TOON (Token-Oriented Object Notation) - một format tiết kiệm token.
Nếu thấy ```toon block, dữ liệu sẽ có dạng bảng với các trường: id, taggedSymbols, title, content.

Nhiệm vụ của bạn:
1. Đọc và hiểu dữ liệu từ format TOON hoặc text thông thường
2. Tóm tắt từng tin tức một cách ngắn gọn, súc tích
3. QUAN TRỌNG NHẤT: Xác định chính xác MÃ CỔ PHIẾU liên quan

Quy tắc nhận diện mã cổ phiếu:
1. Ưu tiên sử dụng mã từ taggedSymbols nếu có và chính xác
2. Nếu taggedSymbols trống/sai, TỰ NHẬN DIỆN từ nội dung:
   - Tìm TÊN CÔNG TY (VD: FPT, Vietcombank, Vingroup, Hòa Phát...)
   - Chuyển thành MÃ CHỨNG KHOÁN chính xác (FPT→FPT, Vietcombank→VCB, Vingroup→VIC, Hòa Phát→HPG)
   - Nhiều công ty → liệt kê TẤT CẢ mã, cách nhau bởi dấu phẩy
3. Chỉ nói ngành/lĩnh vực chung → để trống mã
4. MÃ phải viết HOA và chính xác theo HOSE/HNX/UPCOM

Danh sách công ty phổ biến:
FPT→FPT | Vietcombank→VCB | Vingroup/Vinhomes→VIC,VHM | Hòa Phát→HPG | PV Gas→GAS
Masan→MSN,MCH | Mobile World→MWG | SSI→SSI | Viettel→VTP,CTR,VGI | Sabeco→SAB
Vinamilk→VNM | Techcombank→TCB | BIDV→BID | VietinBank→CTG | VPBank→VPB

Dữ liệu đầu vào:
{articles_list}

Format đầu ra (MỖI TIN MỘT DÒNG):
[MÃ hoặc trống]: [Tiêu đề ngắn] | [Tóm tắt chi tiết với số liệu]

Ví dụ:
FPT: FPT mua lại 5 triệu cổ phiếu quỹ | FPT Corporation mua tối đa 5 triệu cp quỹ Q1/2025, giá ~120K đồng/cp, tổng 600 tỷ đồng, tối ưu vốn.
VCB: Vietcombank lãi Q4 tăng 18% | VCB lợi nhuận Q4/2024 đạt 9.200 tỷ (+18% YoY), tín dụng +15%, dịch vụ +22%.
HPG, VHM: Thép và BĐS hưởng lợi gói 120K tỷ | Gói kích cầu 120K tỷ tập trung hạ tầng, nhà XH; HPG nhu cầu thép +12%, VHM dự án +30%.
: VN-Index tăng nhờ dòng tiền ngoại | VN-Index +8.5 điểm lên 1.245, thanh khoản 18.5K tỷ, ngoại mua ròng 450 tỷ.

Lưu ý:
- Mỗi tin MỘT DÒNG, không xuống dòng
- Không dùng ":" trong nội dung tóm tắt
- GHI RÕ số liệu (%, số tiền, số lượng...)
- Nhiều mã → ghi hết, cách bằng dấu phẩy
- Không xác định được mã → để trống
- Tổng 20 tin quan trọng nhất
- Ưu tiên tin ảnh hưởng lớn

Tóm tắt:
""")

In [30]:
chain_summary = news_summarize_with_stock_template | model
chain_summary_more_temp = news_summarize_with_stock_template | model_more_temperature
chain_summary_pro = news_summarize_with_stock_template | model_pro

print("Đã khởi tạo chains")

Đã khởi tạo chains


In [31]:
try:
    import importlib.util
    if importlib.util.find_spec("toon_format") is None:
        print("Đang cài đặt TOON format...")
        subprocess.check_call(["pip", "install", "toon-format"], stdout=subprocess.DEVNULL)
        print("Đã cài đặt TOON format")
    else:
        print("TOON format đã sẵn")
except Exception as e:
    print(f"Không thể cài đặt TOON: {e}")
    print("Sẽ dùng JSON thông thường")

TOON format đã sẵn


### 💡 Về TOON Format

**TOON (Token-Oriented Object Notation)** là format được thiết kế đặc biệt cho LLM:

- ✅ **Tiết kiệm 30-60% token** so với JSON cho dữ liệu dạng bảng
- ✅ **Dễ đọc** hơn JSON, tương tự YAML
- ✅ **LLM hiểu tốt hơn** nhờ cấu trúc rõ ràng
- ✅ **Giảm chi phí API** đáng kể

**So sánh:**

```json
// JSON: 108 tokens
{
  "users": [
    { "id": 1, "name": "Alice", "role": "admin" },
    { "id": 2, "name": "Bob", "role": "user" }
  ]
}
```

```toon
// TOON: 72 tokens (giảm 33%)
users[2]{id	name	role}:
  1	Alice	admin
  2	Bob	user
```

In [32]:
# Demo: So sánh TOON vs JSON
def demo_toon_savings():
    """
    Demo trực quan về tiết kiệm token của TOON
    """
    sample_data = {
        "articles": [
            {"id": 1, "symbol": "FPT", "title": "FPT công bố kết quả kinh doanh", "price": 120000},
            {"id": 2, "symbol": "VCB", "title": "Vietcombank tăng trưởng tín dụng", "price": 95000},
            {"id": 3, "symbol": "HPG", "title": "Hòa Phát mở rộng sản xuất thép", "price": 28500}
        ]
    }
    
    # JSON format
    json_str = json.dumps(sample_data, ensure_ascii=False, indent=2)
    json_tokens = len(json_str.encode('utf-8')) // 4  # Ước lượng đơn giản
    
    # TOON format
    toon_str = to_toon_format(sample_data)
    toon_tokens = len(toon_str.encode('utf-8')) // 4
    
    # Hiển thị kết quả
    print("=" * 70)
    print("SO SÁNH TOON vs JSON")
    print("=" * 70)
    
    print("\n📄 JSON FORMAT:")
    print(f"   Độ dài: {len(json_str)} ký tự")
    print(f"   Ước lượng: ~{json_tokens} tokens")
    print(f"\n{json_str[:200]}...")
    
    print("\n📋 TOON FORMAT:")
    print(f"   Độ dài: {len(toon_str)} ký tự")
    print(f"   Ước lượng: ~{toon_tokens} tokens")
    print(f"\n{toon_str[:200]}...")
    
    savings = ((json_tokens - toon_tokens) / json_tokens * 100) if json_tokens > 0 else 0
    print(f"\n💰 TIẾT KIỆM: ~{savings:.1f}% tokens")
    print("=" * 70)

# Chạy demo (bỏ comment dòng dưới để xem)
# demo_toon_savings()

In [33]:
# Hàm tiện ích với retry logic và API key rotation
MAX_RETRIES = 5
BASE_DELAY = 30

def switch_api_key():
    """Chuyển sang API key tiếp theo trong danh sách"""
    global current_key_index, current_api_key, model, model_more_temperature, model_pro
    
    if len(GOOGLE_API_KEYS) <= 1:
        return False
    
    current_key_index = (current_key_index + 1) % len(GOOGLE_API_KEYS)
    current_api_key = GOOGLE_API_KEYS[current_key_index]
    
    # Cấu hình lại genai
    genai.configure(api_key=current_api_key)
    
    # Tạo lại models với key mới
    model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.02)
    model_more_temperature = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.1)
    model_pro = ChatGoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25", temperature=0.1)
    
    print(f"Đã chuyển sang API key: ...{current_api_key[-8:]}")
    return True

def invoke_chain_with_retry(chain, prompt, max_retries=MAX_RETRIES, base_delay=BASE_DELAY):
    """Gọi chain với retry và auto-switch API key khi gặp lỗi"""
    retry_count = 0
    key_switched = False
    
    while True:
        try:
            response = chain.invoke(prompt)
            return response
        except Exception as e:
            error_msg = str(e).lower()
            
            # Nếu lỗi quota/rate limit và chưa switch key, thử switch
            if not key_switched and ("quota" in error_msg or "rate" in error_msg or "429" in error_msg):
                if switch_api_key():
                    key_switched = True
                    print("Thử lại với key mới...")
                    continue
            
            retry_count += 1
            if retry_count > max_retries:
                print(f"Đã thử {max_retries} lần nhưng vẫn lỗi: {e}")
                return None
                
            delay = base_delay * (2 ** (retry_count - 1))
            print(f"Lỗi: {e}. Thử lại sau {delay}s (Lần {retry_count}/{max_retries})")
            time.sleep(delay)

def to_toon_format(data):
    """Chuyển dữ liệu sang TOON format"""
    try:
        import subprocess
        json_data = json.dumps(data, ensure_ascii=False)
        
        result = subprocess.run(
            ['npx', '-y', '@toon-format/cli', '--delimiter', '\t'],
            input=json_data,
            capture_output=True,
            text=True,
            encoding='utf-8'
        )
        
        if result.returncode == 0 and result.stdout:
            return result.stdout.strip()
        else:
            return json.dumps(data, ensure_ascii=False, indent=2)
    except Exception as e:
        return json.dumps(data, ensure_ascii=False, indent=2)

def combine_articles(df: pd.DataFrame, use_toon=True) -> str:
    """
    Kết hợp các bài báo thành chuỗi văn bản tối ưu cho LLM
    Cấu trúc all_news.csv: postID, date, sentiment, taggedSymbols, combinedContent
    """
    if use_toon and len(df) > 0:
        try:
            articles_data = []
            for idx, row in df.iterrows():
                combined = str(row.get('combinedContent', ''))
                
                # Tách title từ combinedContent
                title = ""
                content = combined
                
                if "Title:" in combined:
                    parts = combined.split("Description:", 1)
                    title = parts[0].replace("Title:", "").strip()
                    if len(parts) > 1:
                        content = parts[1].strip()
                    title = parts[0].replace("Title:", "").strip()
                    if len(parts) > 1:
                        content = parts[1].strip()[:500]
                
                article = {
                    "id": int(row.get('postID', idx + 1)),
                    "taggedSymbols": str(row.get('taggedSymbols', '[]')),
                    "title": title if title else content[:100],  # Dùng 100 ký tự đầu nếu không có title
                    "content": content
                }
                articles_data.append(article)
            
            # Chuyển sang TOON format
            toon_data = to_toon_format({"articles": articles_data})
            
            # Thêm header giải thích
            return f"```toon\n{toon_data}\n```\n\nLưu ý: Dữ liệu dùng TOON format (tiết kiệm token). Mỗi dòng: id, taggedSymbols, title, content."
            
        except Exception as e:
            print(f"⚠ Không thể dùng TOON, dùng format thông thường: {e}")
    
    # Fallback: format text thông thường
    articles = []
    for idx, row in df.iterrows():
        tagged_symbols = row.get('taggedSymbols', '[]')
        combined = str(row.get('combinedContent', ''))
        
        article_text = f"Tin {idx+1} (postID: {row.get('postID', 'N/A')}):\n"
        article_text += f"Tagged Symbols: {tagged_symbols}\n"
        article_text += f"Nội dung: {combined[:500]}...\n"  # Giới hạn độ dài
        articles.append(article_text)
    
    return "\n---\n".join(articles)

def parse_summary_response(response, date, starting_index=1):
    """
    Phân tích kết quả tóm tắt từ LLM theo định dạng [MÃ]: [Tiêu đề] | [Nội dung]
    """
    pattern = r'^([A-Z, ]*?):\s*(.+?)\s*\|\s*(.+)$'
    response_text = str(response.content)
    lines = response_text.splitlines()
    articles = []

    for line in lines:
        line = line.strip()
    """Phân tích kết quả tóm tắt từ LLM"""
    pattern = r'^([A-Z, ]*?):\s*(.+?)\s*\|\s*(.+)$'
    response_text = str(response.content)
    lines = response_text.splitlines()
    articles = []

    for line in lines:
        line = line.strip()
        if not line:
            continue
        match = re.match(pattern, line)
        if match:
            stock_codes, title, description = match.groups()
            
            # Xử lý mã cổ phiếu: loại bỏ khoảng trắng thừa
            stock_codes = stock_codes.strip()
            
            articles.append({
                "postID": starting_index,
                "stockCodes": stock_codes if stock_codes else "",
                "title": title.strip(),                
                "description": description.strip(),                
                "date": date            
                })            
            starting_index += 1        
            return pd.DataFrame(articles)
        print("✓ Đã định nghĩa các hàm tiện ích với TOON support")
        print("Đã định nghĩa các hàm tiện ích")

In [34]:
# Hàm chính để tóm tắt tin tức với TOON format support
def summarize_news_with_stock_codes(
    csv_path="all_news/all_news.csv",
    output_path="summarized_news_with_stocks.csv",
    batch_size=5,
    start_date=None,
    end_date=None,
    max_articles_per_day=50,
    use_toon=True  # Bật TOON format để tiết kiệm token
):
    """
    Tóm tắt tin tức từ all_news.csv với tự động nhận diện mã cổ phiếu
    
    Parameters:
    -----------
    csv_path : str
        Đường dẫn đến file CSV chứa tin tức gốc
    output_path : str
        Đường dẫn file CSV kết quả
    batch_size : int
        Số ngày xử lý mỗi batch (để lưu checkpoint)
    start_date : str, optional
        Ngày bắt đầu (format: YYYY-MM-DD), nếu None sẽ xử lý từ đầu
    end_date : str, optional
        Ngày kết thúc (format: YYYY-MM-DD), nếu None sẽ xử lý đến cuối
    max_articles_per_day : int
        Số lượng bài báo tối đa mỗi ngày (để tránh vượt quá context length)
    use_toon : bool
        Sử dụng TOON format để tiết kiệm 30-60% token (mặc định: True)
    """
    print(f"📖 Đang đọc dữ liệu từ {csv_path}...")
    
    # Đọc dữ liệu
    df = pd.read_csv(csv_path)
    print(f"✓ Đã đọc {len(df)} bài báo")
    
    # Chuyển đổi cột date sang datetime (cột đã tên là 'date' sẵn)
    df['parsed_date'] = pd.to_datetime(df['date'])
    df['only_date'] = df['parsed_date'].dt.date
    
    # Lọc theo khoảng thời gian nếu được chỉ định
    if start_date:
        start_dt = pd.to_datetime(start_date).date()
        df = df[df['only_date'] >= start_dt]
        print(f"✓ Lọc từ ngày {start_date}: còn {len(df)} bài")
    
    if end_date:
        end_dt = pd.to_datetime(end_date).date()
        df = df[df['only_date'] <= end_dt]
        print(f"✓ Lọc đến ngày {end_date}: còn {len(df)} bài")
    
    # Sắp xếp theo thời gian tăng dần
    df = df.sort_values('date')
    
    # Khởi tạo DataFrame kết quả
    df_summarized = pd.DataFrame(columns=["postID", "stockCodes", "title", "description", "date"])
    
    # Nhóm theo ngày
    date_groups = df.groupby("only_date")
    total_groups = len(date_groups)
    
    print(f"\n📅 Sẽ xử lý {total_groups} ngày")
    print(f"⚙ Batch size: {batch_size} ngày/batch")
    print(f"⚙ Tối đa {max_articles_per_day} bài/ngày")
    print(f"⚙ TOON format: {'✓ BẬT (tiết kiệm 30-60% token)' if use_toon else '✗ TẮT'}\n")
    
    # Kiểm tra checkpoint hiện có
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        if len(existing_df) > 0:
            last_date = pd.to_datetime(existing_df.iloc[-1]['date']).date()
            print(f"🔄 Phát hiện checkpoint: tiếp tục từ ngày {last_date}")
            df_summarized = existing_df
            # Bỏ qua các ngày đã xử lý
            date_groups = [(d, g) for d, g in date_groups if d > last_date]
            total_groups = len(date_groups)
            print(f"✓ Còn {total_groups} ngày cần xử lý\n")
    
    # Xử lý theo batch
    current_idx = len(df_summarized) + 1
    
    for batch_idx in range(0, total_groups, batch_size):
        batch_end = min(batch_idx + batch_size, total_groups)
        batch_dates = list(date_groups)[batch_idx:batch_end]
        
        print(f"\n{'='*60}")
        print(f"📦 BATCH {batch_idx//batch_size + 1}: Xử lý ngày {batch_idx+1} đến {batch_end}/{total_groups}")
        print(f"{'='*60}")
        
        batch_summaries = []
        
        for i, (date, group) in enumerate(batch_dates):
            date_position = batch_idx + i + 1
            
            # Giới hạn số lượng bài báo mỗi ngày
            if len(group) > max_articles_per_day:
                print(f"⚠ Ngày {date} có {len(group)} bài, giới hạn xuống {max_articles_per_day} bài")
                group = group.head(max_articles_per_day)
            
            # Kết hợp các bài báo (với TOON nếu được bật)
            combined_articles = combine_articles(group, use_toon=use_toon)
            
            # Tạo prompt
            summary_prompt = {
                "articles_list": combined_articles
            }
            
            print(f"\n[{date_position}/{total_groups}] 📰 Xử lý ngày {date} ({len(group)} bài báo)...")
            
            # Retry với nhiều lần thử
            max_summary_retries = 10
            summary_retry_count = 0
            summary_df = pd.DataFrame()
            
            while summary_retry_count < max_summary_retries:
                # Chọn model dựa trên số lần retry
                if summary_retry_count < 3:
                    selected_chain = chain_summary
                    model_name = "flash-lite (temp=0.02)"
                elif summary_retry_count < 5:
                    selected_chain = chain_summary_more_temp
                    model_name = "flash-lite (temp=0.1)"
                else:
                    selected_chain = chain_summary_pro
                    model_name = "pro (temp=0.1)"
                
                print(f"   🤖 Model: {model_name} | Format: {'TOON' if use_toon else 'Text'}")
                
                # Gọi API với retry logic
                summary_response = invoke_chain_with_retry(selected_chain, summary_prompt)
                time.sleep(1)  # Rate limiting
                
                if summary_response is None:
                    print(f"   ❌ Không thể tóm tắt ngày {date}")
                    break
                
                # Parse kết quả
                date_str = f"{date}T00:00:00+07:00"
                summary_df = parse_summary_response(summary_response, date_str, starting_index=current_idx)
                
                # Kiểm tra kết quả
                if len(summary_df) > 0:
                    print(f"   ✓ Tóm tắt thành {len(summary_df)} tin")
                    break
                
                summary_retry_count += 1
                print(f"   ⚠ Kết quả rỗng, thử lại lần {summary_retry_count}/{max_summary_retries}")
                time.sleep(BASE_DELAY)
            
            # Thêm vào kết quả batch
            if len(summary_df) > 0:
                current_idx += len(summary_df)
                batch_summaries.append(summary_df)
            else:
                print(f"   ❌ Thất bại sau {max_summary_retries} lần thử cho ngày {date}")
        
        # Lưu checkpoint sau mỗi batch
        if batch_summaries:
            batch_df = pd.concat(batch_summaries, ignore_index=True)
            df_summarized = pd.concat([df_summarized, batch_df], ignore_index=True)
            
            # Lưu file
            df_summarized.to_csv(output_path, index=False, encoding='utf-8-sig')
            
            checkpoint_num = batch_idx // batch_size + 1
            print(f"\n💾 Đã lưu checkpoint {checkpoint_num}: {len(df_summarized)} tin tóm tắt")
    
    # Lưu file cuối cùng
    df_summarized.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\n{'='*60}")
    print(f"✅ HOÀN THÀNH!")
    print(f"📊 Tổng số tin tóm tắt: {len(df_summarized)}")
    print(f"💾 Đã lưu vào: {output_path}")
    if use_toon:
        print(f"💰 Ước tính tiết kiệm: 30-60% token nhờ TOON format")
    print(f"{'='*60}")
    
    return df_summarized

print("✓ Đã định nghĩa hàm summarize_news_with_stock_codes() với TOON support")

✓ Đã định nghĩa hàm summarize_news_with_stock_codes() với TOON support


## 🚀 Chạy tóm tắt tin tức

### Lợi ích khi dùng TOON:
- 💰 **Giảm 30-60% chi phí API** (ít token hơn)
- ⚡ **Phản hồi nhanh hơn** (input nhỏ gọn)
- 🎯 **LLM hiểu tốt hơn** (cấu trúc rõ ràng)

### Cách sử dụng:

In [ ]:
# Chạy tóm tắt tin tức với TOON format
# TOON giúp giảm 30-60% token → Tiết kiệm chi phí API đáng kể!

result_df = summarize_news_with_stock_codes(
    csv_path="all_news/all_news.csv",
    output_path="summarized_news_with_stocks.csv",
    batch_size=5,
    start_date="2024-01-01",
    end_date=None,
    use_toon=True
)

📖 Đang đọc dữ liệu từ all_news/all_news.csv...
✓ Đã đọc 241990 bài báo
✓ Đã đọc 241990 bài báo
✓ Lọc từ ngày 2024-01-01: còn 152334 bài

📅 Sẽ xử lý 670 ngày
⚙ Batch size: 5 ngày/batch
⚙ Tối đa 50 bài/ngày
⚙ TOON format: ✓ BẬT (tiết kiệm 30-60% token)

🔄 Phát hiện checkpoint: tiếp tục từ ngày 2024-07-13
✓ Còn 475 ngày cần xử lý


📦 BATCH 1: Xử lý ngày 1 đến 5/475
⚠ Ngày 2024-07-14 có 168 bài, giới hạn xuống 50 bài

[1/475] 📰 Xử lý ngày 2024-07-14 (50 bài báo)...
   🤖 Model: flash-lite (temp=0.02) | Format: TOON
✓ Lọc từ ngày 2024-01-01: còn 152334 bài

📅 Sẽ xử lý 670 ngày
⚙ Batch size: 5 ngày/batch
⚙ Tối đa 50 bài/ngày
⚙ TOON format: ✓ BẬT (tiết kiệm 30-60% token)

🔄 Phát hiện checkpoint: tiếp tục từ ngày 2024-07-13
✓ Còn 475 ngày cần xử lý


📦 BATCH 1: Xử lý ngày 1 đến 5/475
⚠ Ngày 2024-07-14 có 168 bài, giới hạn xuống 50 bài

[1/475] 📰 Xử lý ngày 2024-07-14 (50 bài báo)...
   🤖 Model: flash-lite (temp=0.02) | Format: TOON
Lỗi: Invalid argument provided to Gemini: 400 API key not valid

### ⚙️ Các tùy chọn nâng cao

Bạn có thể tùy chỉnh các tham số:

| Tham số | Mô tả | Giá trị mặc định |
|---------|-------|------------------|
| `use_toon` | Bật TOON format (tiết kiệm 30-60% token) | `True` |
| `batch_size` | Số ngày xử lý mỗi lần (checkpoint) | `5` |
| `max_articles_per_day` | Giới hạn bài báo/ngày | `50` |
| `start_date` | Ngày bắt đầu (YYYY-MM-DD) | `None` (từ đầu) |
| `end_date` | Ngày kết thúc (YYYY-MM-DD) | `None` (đến cuối) |

**Khi nào nên tắt TOON?**
- Dữ liệu không đồng nhất (mỗi bài có cấu trúc khác nhau)
- Đã quen với JSON format
- Debugging (JSON dễ đọc hơn)

## Xem kết quả

Xem một số mẫu tin tức đã tóm tắt:

In [ ]:
# Xem kết quả tóm tắt
print(f"Tổng số tin đã tóm tắt: {len(result_df)}")
print(f"\nCác cột: {result_df.columns.tolist()}")
print(f"\n{'='*80}")
print("MẪU 10 TIN TÓM TẮT ĐẦU TIÊN:")
print(f"{'='*80}\n")

for idx, row in result_df.head(10).iterrows():
    stock_codes = row['stockCodes'] if row['stockCodes'] else "Không có mã"
    print(f"[{row['postID']}] {stock_codes}")
    print(f"📅 {row['date']}")
    print(f"📌 {row['title']}")
    print(f"📝 {row['description']}")
    print(f"{'-'*80}\n")

In [ ]:
# Phân tích thống kê mã cổ phiếu
print("📊 THỐNG KÊ MÃ CỔ PHIẾU ĐƯỢC ĐỀ CẬP\n")

# Đếm số lượng tin có mã và không có mã
has_stock = result_df[result_df['stockCodes'] != ''].shape[0]
no_stock = result_df[result_df['stockCodes'] == ''].shape[0]

print(f"✓ Tin có mã cổ phiếu: {has_stock} ({has_stock/len(result_df)*100:.1f}%)")
print(f"✓ Tin không có mã: {no_stock} ({no_stock/len(result_df)*100:.1f}%)")

# Tách các mã cổ phiếu và đếm tần suất
all_stocks = []
for codes in result_df['stockCodes']:
    if codes:
        # Tách các mã bằng dấu phẩy
        stocks = [s.strip() for s in codes.split(',') if s.strip()]
        all_stocks.extend(stocks)

if all_stocks:
    from collections import Counter
    stock_counter = Counter(all_stocks)
    
    print(f"\n📈 TOP 20 MÃ CỔ PHIẾU ĐƯỢC ĐỀ CẬP NHIỀU NHẤT:\n")
    for i, (stock, count) in enumerate(stock_counter.most_common(20), 1):
        print(f"{i:2d}. {stock:6s} - {count:4d} lần")
else:
    print("\n⚠ Chưa có mã cổ phiếu nào được nhận diện")

## Các tính năng nâng cao

### Tìm kiếm tin tức theo mã cổ phiếu

In [ ]:
# Tìm kiếm tin tức theo mã cổ phiếu ( sau này dùng thì có cái mà dùng )
def search_by_stock_code(df, stock_code):
    """
    Tìm tất cả tin tức liên quan đến một mã cổ phiếu
    """
    stock_code = stock_code.upper().strip()
    mask = df['stockCodes'].str.contains(stock_code, case=False, na=False)
    results = df[mask]
    
    print(f"🔍 Tìm thấy {len(results)} tin tức về mã {stock_code}\n")
    print(f"{'='*80}")
    
    for idx, row in results.iterrows():
        print(f"\n[{row['postID']}] {row['stockCodes']}")
        print(f"📅 {row['date']}")
        print(f"📌 {row['title']}")
        print(f"📝 {row['description']}")
        print(f"{'-'*80}")
    
    return results

# Ví dụ: Tìm tin về FPT
# search_by_stock_code(result_df, "FPT")